# Twitter Data Notebook

This notebook preps the scraped twitter data for sentiment classification

## Modules

In [1]:
import pandas as pd
from datetime import datetime
from clean_tweets import clean_tweet, tweet_filtering
import string
import dataframe_image as dfi
from difflib import SequenceMatcher

## Data Cleaning (Twitter API)

In this section we clean the data scraped from  the twitter API

### Initial Data Summary

In this section, we produce an initial data summary for a sanity check.  

In [2]:
def parse_datetime(dt_str: str):
    """
    This function parses the datetime of a string into a python datetime
    dt_str: Raw datetime string
    returns: Datetime as a datetime type variable
    """
    return datetime.strptime(dt_str,'%a %b %d %H:%M:%S +0000 %Y').date()

In [3]:
# Defining list of target companies
screen_names = ['AdaniOnline', 'Microsoft', 'FTX_Official', 'Meta', 'AirCanada', 'fia']

In [4]:
# Initializing list to hold results
data_all = []

# Iterating through each company
for screen_name in screen_names:
    data = {}
    
    # Reading in the tweets
    df = pd.read_csv(f"data/{screen_name}_tweets.csv", index_col=0)
    
    # Parsing the date
    df['date'] = df['date'].apply(lambda date: parse_datetime(date))
    df['date'] = pd.to_datetime(df['date'] )
    
    # Creating summary data
    data['Twitter Handle'] = screen_name
    data['Start Date'] = df['date'].min()
    data['End Date'] = df['date'].max()
    data['Number of Tweets'] = len(df)
    
    data_all.append(data)

In [5]:
data_summary_api = pd.DataFrame(data_all)
data_summary_api

,Twitter Handle,Start Date,End Date,Number of Tweets
0,AdaniOnline,2023-03-06,2023-03-15,255
1,Microsoft,2023-03-05,2023-03-15,4775
2,FTX_Official,2023-03-06,2023-03-15,51
3,Meta,2023-03-05,2023-03-15,831
4,AirCanada,2023-03-06,2023-03-16,864
5,fia,2023-03-06,2023-03-16,602


In [6]:
print("Total Number of Tweets: ", data_summary_api["Number of Tweets"].sum())

Total Number of Tweets:  7378


### Cleaning Data

In this section we actually clean the data

In [7]:
data_all = []
api_df = []
for screen_name in screen_names:
    data = {}
    
    # Reading in the tweets
    df = pd.read_csv(f"data/{screen_name}_tweets.csv", index_col=0)
    
    # Removing tweets with non-ascii characters (i.e. emogies)
    df['filter'] = df['text'].apply(lambda text: tweet_filtering(text))
    df = df[df['filter'] == False]
    
    # Cleaning the tweets
    df['text'] = df['text'].apply(lambda text: clean_tweet(text))
    
    # Removing empty tweets
    df = df[df['text'] != ""]
    
    # Parsing the date
    df['date'] = df['date'].apply(lambda date: parse_datetime(date))
    df['date'] = pd.to_datetime(df['date'] )
    
    df["twitter_handle"] = screen_name
    
    api_df.append(df)

    # Creating summary data
    data['Twitter Handle'] = screen_name
    data['Start Date'] = df['date'].min()
    data['End Date'] = df['date'].max()
    data['Number of Tweets'] = len(df)
    
    data_all.append(data)
    
    df.to_csv(f"data/{screen_name}_tweets_cleaned.csv")

In [8]:
api_df = pd.concat(api_df)

In [9]:
data_summary_api_cleaned = pd.DataFrame(data_all)
data_summary_api_cleaned

,Twitter Handle,Start Date,End Date,Number of Tweets
0,AdaniOnline,2023-03-06,2023-03-15,34
1,Microsoft,2023-03-05,2023-03-15,2964
2,FTX_Official,2023-03-06,2023-03-15,10
3,Meta,2023-03-06,2023-03-15,205
4,AirCanada,2023-03-06,2023-03-16,259
5,fia,2023-03-06,2023-03-16,191


In [10]:
print("Total Number of Tweets: ", data_summary_api_cleaned["Number of Tweets"].sum())

Total Number of Tweets:  3663


## Data Cleaning (Selenium)

In this section we clean the data scraped from Twitter using Selenium

### Initial Data Summary

In this section, we produce an initial data summary for a sanity check.  

In [11]:
selenium_scraped = pd.read_csv(
    "data/tweet_with_replies_all.csv",
   header=None
)

selenium_scraped.columns = columns=["twitter_handle", "text", "parent_id", "date"]

In [12]:
selenium_scraped = selenium_scraped[selenium_scraped["twitter_handle"].isin(['AdaniOnline', 'Microsoft', 'FTX_Official', 'Meta', 'AirCanada', 'fia'])]

In [13]:
twitter_handles = selenium_scraped["twitter_handle"].unique()

selenium_scraped['date'] = selenium_scraped['date'].apply(lambda date: parse_datetime(date))

data_all = []
for twitter_handle in twitter_handles:
    
    data = {}
    
    df = selenium_scraped[selenium_scraped["twitter_handle"] == twitter_handle]
    
    
    # Creating summary data
    data['Twitter Handle'] = twitter_handle
    data['Start Date'] = df['date'].min()
    data['End Date'] = df['date'].max()
    data['Number of Tweets'] = len(df)
    
    data_all.append(data)

In [14]:
selenium_summary = pd.DataFrame(data_all)
selenium_summary

,Twitter Handle,Start Date,End Date,Number of Tweets
0,Microsoft,2022-10-17,2023-03-15,1252
1,AirCanada,2022-09-27,2023-03-13,1392
2,Meta,2020-01-14,2023-02-27,1265
3,fia,2021-08-15,2023-03-14,4554


In [15]:
print("Total Number of Tweets: ", selenium_summary["Number of Tweets"].sum())

Total Number of Tweets:  8463


### Cleaning Data

In this section we actually clean the data

In [16]:
# Removing empty tweets
selenium_scraped = selenium_scraped[selenium_scraped['text'].isna() == False]
selenium_scraped = selenium_scraped[selenium_scraped['text'] != ""]

# Removing tweets with non-ascii characters (i.e. emogies)
selenium_scraped['filter'] = selenium_scraped['text'].apply(lambda text: tweet_filtering(text))

selenium_scraped = selenium_scraped[selenium_scraped['filter'] == False]

# Cleaning the tweets
selenium_scraped['text'] = selenium_scraped['text'].apply(lambda text: clean_tweet(text))

# Removing empty tweets
selenium_scraped = selenium_scraped[selenium_scraped['text'] != ""]

# Parsing the data
# selenium_scraped['date'] = selenium_scraped['date'].apply(lambda date: parse_datetime(date))
selenium_scraped['date'] = pd.to_datetime(selenium_scraped['date'] )

In [17]:
selenium_scraped.to_csv("results/tweet_with_replies_all_cleaned.csv")

In [18]:
twitter_handles = selenium_scraped["twitter_handle"].unique()

data_all = []
for twitter_handle in twitter_handles:
    
    data = {}
    
    df = selenium_scraped[selenium_scraped["twitter_handle"] == twitter_handle]
    
    # Creating summary data
    data['Twitter Handle'] = twitter_handle
    data['Start Date'] = df['date'].min()
    data['End Date'] = df['date'].max()
    data['Number of Tweets'] = len(df)
    
    data_all.append(data)

In [19]:
selenium_summary_cleaned = pd.DataFrame(data_all)
selenium_summary_cleaned

,Twitter Handle,Start Date,End Date,Number of Tweets
0,Microsoft,2022-10-17,2023-03-15,1058
1,AirCanada,2022-09-27,2023-03-13,983
2,Meta,2020-01-14,2023-02-27,906
3,fia,2021-08-15,2023-03-14,3599


In [20]:
print("Total Number of Tweets: ", selenium_summary_cleaned["Number of Tweets"].sum())

Total Number of Tweets:  6546


## Initial Comparison (i.e. Uncleaned)

In this section, we produce a table to compare the two datasets (uncleaned versions)

In [21]:
# Merging the two datasets together
data_merged = pd.merge(left=data_summary_api, right=selenium_summary, how='outer', on="Twitter Handle", suffixes=(' API', ' Selenium'))

In [22]:
# Setting the index
data_merged.set_index("Twitter Handle", inplace=True)

In [23]:
data_merged["Start Date API"] = data_merged["Start Date API"].astype(str)
data_merged["End Date API"] = data_merged["Start Date API"].astype(str)

In [24]:
data_merged['Number of Tweets API'] = data_merged['Number of Tweets API'].fillna(0).astype(int)
data_merged['Number of Tweets Selenium'] = data_merged['Number of Tweets Selenium'].fillna(0).astype(int)

In [25]:
# Creating multi-indexed columns
columns = list(data_summary_api.columns[1:]) + list(data_summary_api.columns[1:])

types = ["API" for i in range(0,3)] + ["Selenium" for i in range(0,3)]

multi_index_columns = list(zip(types, columns))

data_merged.columns = pd.MultiIndex.from_tuples(multi_index_columns)

In [26]:
data_merged

API                                 Selenium  \
                Start Date    End Date Number of Tweets  Start Date   
Twitter Handle                                                        
AdaniOnline     2023-03-06  2023-03-06              255         NaN   
Microsoft       2023-03-05  2023-03-05             4775  2022-10-17   
FTX_Official    2023-03-06  2023-03-06               51         NaN   
Meta            2023-03-05  2023-03-05              831  2020-01-14   
AirCanada       2023-03-06  2023-03-06              864  2022-09-27   
fia             2023-03-06  2023-03-06              602  2021-08-15   

                                             
                  End Date Number of Tweets  
Twitter Handle                               
AdaniOnline            NaN                0  
Microsoft       2023-03-15             1252  
FTX_Official           NaN                0  
Meta            2023-02-27             1265  
AirCanada       2023-03-13             1392  
fia             2023-03-14             4554

In [27]:
# Replacing some empty values
data_merged = data_merged.fillna("")
data_merged = data_merged.replace("NaT", "")
data_merged = data_merged.replace(0, "")

In [28]:
# Setting styles and writing to disk
data_merged = data_merged.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
data_merged.set_properties(**{'text-align': 'center'})
dfi.export(data_merged, 'results/twitter_data_merged.png')

[0330/094254.146198:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0330/094254.148220:WARNING:sandbox_linux.cc(393)] InitializeSandbox() called with multiple threads in process gpu-process.
[0330/094254.255960:INFO:headless_shell.cc(107)] 50413 bytes written to file /tmp/tmp1bz9cva3/temp.png


## Data Merging

In this section we merge the two data sets into a single set

In [28]:
def similar(a: str, b: str):
    # CITATION: https://stackoverflow.com/questions/17388213/find-the-similarity-metric-between-two-strings
    """
    This function compares two strings to determine their similarity
    a: string 1
    b: string 2
    return: Similarity measure between 0 and 1
    """
    return SequenceMatcher(lambda x: x == " ", a, b).ratio()

In [29]:
# Dropping unnessecary column
selenium_scraped = selenium_scraped.drop(['parent_id'], axis=1)

In [30]:
# Dropping unnessecary column
api_df = api_df.drop(['id'], axis=1)

In [31]:
# We merge both datasets
df_merged = pd.concat([selenium_scraped, api_df])

In [32]:
# Reset the index
df_merged = df_merged.reset_index(drop=True)

In [33]:
df_merged

,twitter_handle,text,date,filter
0,Microsoft,hello,2023-03-15,False
1,Microsoft,don't you think that chatgpt 3 is biased,2023-03-15,False
2,Microsoft,takes us back,2023-03-14,False
3,Microsoft,my xbox since 2003,2023-03-14,False
4,Microsoft,still runs like a charm,2023-03-14,False
...,...,...,...,...
10204,fia,rt,2023-03-07,False
10205,fia,rt,2023-03-07,False
10206,fia,rt something more important,2023-03-07,False
10207,fia,rt,2023-03-07,False


In [34]:
# Getting list of twitter handles
twitter_handles = df_merged["twitter_handle"].unique()

# Creating list to hold unique tweets for each handle
df_duplicate_free = []

# Iterating through each handle
for twitter_handle in twitter_handles:
    print("Filtering:", twitter_handle)

    # Filtering to specific twitter handle
    df = df_merged[df_merged["twitter_handle"] == twitter_handle]

    # We reset the index since so we can use it to filter out comparisons against themselves
    df = df.reset_index()

    # CITATION: https://www.geeksforgeeks.org/python-program-to-perform-cross-join-in-pandas/
    # Creating dummy key
    df['key'] = 1

    # Performing cross join
    df_cross_joined = pd.merge(df, df, on ='key', suffixes=('_L', '_R')).drop("key", 1)

    # Filtering out tweets that aren't from the same time
    df_cross_joined = df_cross_joined[df_cross_joined["date_L"] == df_cross_joined["date_R"]]

    # Filtering out comparisons against themselves
    df_cross_joined = df_cross_joined[df_cross_joined["index_L"] != df_cross_joined["index_R"]]

    # If there are still potential duplicates
    if len(df_cross_joined) > 0:
        # Computing the similarity between each pair of tweets
        df_cross_joined['similarity'] = df_cross_joined.apply(lambda row: similar(row["text_L"], row["text_R"]), axis=1)

        # Filtering to the duplicates
        duplicates = df_cross_joined[df_cross_joined['similarity'] > 0.9]

        # Removing the duplicates
        df[df['index'].isin(duplicates['index_R']) == False]

    # Adding the duplicate free result
    df_duplicate_free.append(df)
    

Filtering: Microsoft


/tmp/ipykernel_69196/3014575831.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_cross_joined = pd.merge(df, df, on ='key', suffixes=('_L', '_R')).drop("key", 1)


Filtering: AirCanada


/tmp/ipykernel_69196/3014575831.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_cross_joined = pd.merge(df, df, on ='key', suffixes=('_L', '_R')).drop("key", 1)


Filtering: Meta


/tmp/ipykernel_69196/3014575831.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_cross_joined = pd.merge(df, df, on ='key', suffixes=('_L', '_R')).drop("key", 1)


Filtering: fia


/tmp/ipykernel_69196/3014575831.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_cross_joined = pd.merge(df, df, on ='key', suffixes=('_L', '_R')).drop("key", 1)


Filtering: AdaniOnline
Filtering: FTX_Official


/tmp/ipykernel_69196/3014575831.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_cross_joined = pd.merge(df, df, on ='key', suffixes=('_L', '_R')).drop("key", 1)
/tmp/ipykernel_69196/3014575831.py:22: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_cross_joined = pd.merge(df, df, on ='key', suffixes=('_L', '_R')).drop("key", 1)


In [35]:
# Merging everything together
df_duplicate_free = pd.concat(df_duplicate_free)

In [37]:
# Resetting the index
df_duplicate_free = df_duplicate_free.reset_index(drop=True)

In [38]:
# Dropping unneeded columns
df_duplicate_free = df_duplicate_free.drop(["filter", "index", "key"],axis=1)

In [53]:
# Writing to disk
df_duplicate_free.to_csv("results/twitter_scraped_cleaned.csv")

## Merged Data Summary

In this section we produce a final summary of the merged data

In [49]:
# Reading in the above data since computing the cleaned data can take a while
df_duplicate_free = pd.read_csv("results/twitter_scraped_cleaned.csv", index_col=0)

In [42]:
# Getting list of the twitter handles
twitter_handles = df_duplicate_free["twitter_handle"].unique()

# Creating list to hold results
data_all = []

# Iterating through each hanlde
for twitter_handle in twitter_handles:
    
    # Initializing dict to hold result
    data = {}
        
    # Filtering to the handle
    df = df_duplicate_free[df_duplicate_free["twitter_handle"] == twitter_handle]
    
    
    # Creating summary data
    data['Twitter Handle'] = twitter_handle
    data['Start Date'] = df['date'].min()
    data['End Date'] = df['date'].max()
    data['Number of Tweets'] = len(df)
    
    # Adding the summary
    data_all.append(data)

In [45]:
# Converting to DF and some basic processing
duplicate_removed_summary = pd.DataFrame(data_all)
duplicate_removed_summary["Start Date"] = duplicate_removed_summary["Start Date"].astype(str)
duplicate_removed_summary["End Date"] = duplicate_removed_summary["End Date"].astype(str)
duplicate_removed_summary.set_index("Twitter Handle", inplace=True)                                    

In [46]:
# We don't want to include these handles
duplicate_removed_summary = duplicate_removed_summary[duplicate_removed_summary.index.isin(['AdaniOnline', 'FTX_Official']) == False]

In [47]:
# Adding sample tweets
duplicate_removed_summary["Example"] = [
    df_duplicate_free.iloc[4,]['text'], 
    df_duplicate_free.iloc[4034,]['text'],
    df_duplicate_free.iloc[5277,]['text'], 
    df_duplicate_free.iloc[6396,]['text']
]

In [50]:
# Performing styling and writing table to png
duplicate_removed_summary = duplicate_removed_summary.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
('max-width', '5')
duplicate_removed_summary = duplicate_removed_summary.set_properties(subset=['Example'], **{'width': '200px'})
duplicate_removed_summary = duplicate_removed_summary.set_properties(**{'text-align': 'center'})
dfi.export(duplicate_removed_summary, 'results/twitter_cleaned.png')

[0330/093525.659324:WARNING:bluez_dbus_manager.cc(247)] Floss manager not present, cannot set Floss enable/disable.
[0330/093525.667176:WARNING:sandbox_linux.cc(393)] InitializeSandbox() called with multiple threads in process gpu-process.
[0330/093525.777998:INFO:headless_shell.cc(107)] 48269 bytes written to file /tmp/tmpret280um/temp.png
